In [17]:
import librosa
import numpy as np
import pandas as pd
import noisereduce as nr
import parselmouth
import textgrid
import os

# 📌 **Dosya Yolları (Manuel ve Doğru)**
wav_file = "C:\\Users\\DELL\\Documents\\GitHub\\fonetika-data\\test-audios-mfa\\test-audios-mfa\\test-audios-mfa\\test-corpus\\vahiy1.wav"
textgrid_file = "C:\\Users\\DELL\\Documents\\GitHub\\fonetika-data\\test-audios-mfa\\test-audios-mfa\\test-audios-mfa\\test-output\\vahiy1.TextGrid"
output_csv = "C:\\Users\\DELL\\Documents\\GitHub\\fonetika-data\\test-audios-mfa\\output_features.csv"

print(f"✅ WAV Dosyası: {wav_file}")
print(f"✅ TextGrid Dosyası: {textgrid_file}")
print(f"✅ Çıktı Kaydedilecek: {output_csv}")

# 📌 **Praat ile Formant Çıkarma**
def get_formants_praat(y, sr):
    """Praat kullanarak F1, F2, F3 formantlarını çıkarır."""
    snd = parselmouth.Sound(y, sr)
    formants = snd.to_formant_burg(time_step=0.01, max_number_of_formants=5, maximum_formant=5500)

    f1_list, f2_list, f3_list = [], [], []

    for t in np.arange(0, snd.duration, 0.01):
        f1 = formants.get_value_at_time(1, t)
        f2 = formants.get_value_at_time(2, t)
        f3 = formants.get_value_at_time(3, t)

        if f1 and 100 < f1 < 1000:
            f1_list.append(f1)
        if f2 and 500 < f2 < 3000:
            f2_list.append(f2)
        if f3 and 1500 < f3 < 4000:
            f3_list.append(f3)

    return (np.mean(f1_list) if f1_list else None, 
            np.mean(f2_list) if f2_list else None, 
            np.mean(f3_list) if f3_list else None)

# 📌 **TextGrid Okuma**
def parse_textgrid(textgrid_path):
    """MFA tarafından üretilmiş TextGrid dosyasını okuyarak fonemleri çıkarır."""
    tg = textgrid.TextGrid()
    tg.read(textgrid_path)
    intervals = []

    for tier in tg:
        for interval in tier:
            phoneme = interval.mark.strip()
            if phoneme:
                intervals.append((phoneme, interval.minTime, interval.maxTime))

    return intervals

# 📌 **Ana İşlem Fonksiyonu**
def process_wav(wav_path, textgrid_path, output_csv):
    y, sr = librosa.load(wav_path, sr=None)
    y_denoised = nr.reduce_noise(y=y, sr=sr)
    intervals = parse_textgrid(textgrid_path)

    data = []
    for phoneme, start, end in intervals:
        start_sample = int(start * sr)
        end_sample = int(end * sr)
        segment = y_denoised[start_sample:end_sample]

        # 📌 Sessiz veya çok kısa segmentleri atla
        duration = end - start
        if len(segment) < 2048 or not np.any(segment) or duration < 0.01:
            print(f"⚠️ Atlandı: {phoneme} ({start}-{end}) sessiz veya çok kısa!")
            continue

        # 📌 MFCC (Sadece ilk 3 bileşen)
        mfccs = librosa.feature.mfcc(y=segment, sr=sr, n_mfcc=3)
        mfcc_means = np.mean(mfccs, axis=1)

        # 📌 Formant Analizi
        F1, F2, F3 = get_formants_praat(segment, sr)

        # 📌 Büyük segmentler için uyarı
        if duration > 0.2:
            print(f"🚨 UYARI: {phoneme} ({start}-{end}) segmenti çok uzun! Formantlar yanlış olabilir.")

        # 📌 Sonuçları ekle
        data.append([phoneme, start, end, duration, mfcc_means[0], mfcc_means[1], mfcc_means[2], F1, F2, F3, os.path.basename(wav_path)])

    # 📌 Sonuçları CSV'ye kaydet
    columns = ["phoneme", "start", "end", "duration", "MFCC1", "MFCC2", "MFCC3", "F1", "F2", "F3", "audio_file"]
    df = pd.DataFrame(data, columns=columns)
    df.to_csv(output_csv, index=False)

    print(f"✅ İşlem tamamlandı! Sonuçlar {output_csv} dosyasına kaydedildi.")

# 📌 **Çalıştır**
process_wav(wav_file, textgrid_file, output_csv)


✅ WAV Dosyası: C:\Users\DELL\Documents\GitHub\fonetika-data\test-audios-mfa\test-audios-mfa\test-audios-mfa\test-corpus\vahiy1.wav
✅ TextGrid Dosyası: C:\Users\DELL\Documents\GitHub\fonetika-data\test-audios-mfa\test-audios-mfa\test-audios-mfa\test-output\vahiy1.TextGrid
✅ Çıktı Kaydedilecek: C:\Users\DELL\Documents\GitHub\fonetika-data\test-audios-mfa\output_features.csv
🚨 UYARI: vahiy (0.37-1.78) segmenti çok uzun! Formantlar yanlış olabilir.
⚠️ Atlandı: v (0.37-0.38) sessiz veya çok kısa!
⚠️ Atlandı: a (0.38-0.41) sessiz veya çok kısa!
🚨 UYARI: h (0.41-1.57) segmenti çok uzun! Formantlar yanlış olabilir.
⚠️ Atlandı: i (1.57-1.61) sessiz veya çok kısa!
✅ İşlem tamamlandı! Sonuçlar C:\Users\DELL\Documents\GitHub\fonetika-data\test-audios-mfa\output_features.csv dosyasına kaydedildi.
